# Ensure proper python libraries installed

In [10]:
import pycaret
print(pycaret.__version__)

3.3.2


In [13]:
import pandas 
print(pandas.__version__)

1.4.4


# Import data

In [17]:
import pandas as pd

# Relative path to the CSV file
trainSet = pd.read_csv('Comment_Sample_With_Stance_Similarity.csv')

# Display the first few rows of the DataFrame
print(trainSet.head())

           Document ID        Posted Date        Submitter  \
0  DEA-2024-0059-32951  2024-07-18T04:00Z    Nathan Fisher   
1   DEA-2024-0059-9342  2024-05-31T04:00Z        Anonymous   
2  DEA-2024-0059-20636  2024-06-18T04:00Z       Lyn Shoots   
3  DEA-2024-0059-22701  2024-06-19T04:00Z           Nate C   
4  DEA-2024-0059-23753  2024-06-21T04:00Z  Carol McFarland   

                                             Comment  \
0  Respectfully, end the Drug War!\n\nTo Whom it ...   
1  I work at a small cannabis business in MI\n\nW...   
2  Official Comment Drug Enforcement Agency ,\n\n...   
3  As a person who has been living with epilepsy ...   
4  Hello. 70 year old retired public schools prin...   

                       Stance  Comment Similarity Score  
0  Supports Beyond Schedule 3                  0.993366  
1  Supports Beyond Schedule 3                  0.131234  
2                    Supports                  0.614912  
3  Supports Beyond Schedule 3                  0.182440  


# Create Class Imbalance Methods List

In [18]:
import re
from pycaret.classification import setup, compare_models, pull

try:
    # Intentionally set an invalid value for fix_imbalance_method
     setup(
        data=trainSet,
        target='Stance',
        session_id=123,
        text_features=['Comment'],
        fix_imbalance=True,
        fix_imbalance_method='invalid_value'
    )
except ValueError as e:
    # Extract the valid options from the error message
    match = re.search(r"Choose from: (.*)\.", str(e))
    if match:
        imbalance_methods = match.group(1).split(', ')
        imbalance_methods = [option.strip() for option in imbalance_methods]
        print("Available fix_imbalance_method options:")
        print(imbalance_methods)


Available fix_imbalance_method options:
['condensednearestneighbour', 'editednearestneighborus', 'repeatededitednearestneighbours', 'allknn', 'instancehardnessthreshold', 'nearmiss', 'neighbourhoodcleaningrule', 'onesidedselection', 'randomundersampler', 'tomeklinks', 'randomoversampler', 'smote', 'smotenc', 'smoten', 'adasyn', 'borderlinesmote', 'kmeanssmote', 'svmsmote', 'smoteenn', 'smotetomek']


# Compare_Models for each Imbalance Method 

In [19]:
# Initialize an empty DataFrame to store the Compare Model results
results = pd.DataFrame()

# Iterate over imbalance methods
for method in imbalance_methods:
    try:
        # Initialize PyCaret setup for classification
        setup(data=trainSet,
                  target='Stance', 
                  session_id=123, 
                  text_features=['Comment'],
                  ignore_features=['Document ID','Posted Date','Submitter','Comment Similarity Score'],
                  fix_imbalance=True,
                  fix_imbalance_method=method, 
                  preprocess=True)

        # Compare different models
        model_results = compare_models()

        model_results = pull()

        # Add the method as a new column
        model_results['method'] = method

        # Append results to the DataFrame
        results = pd.concat([results, model_results], ignore_index=True)

    except TypeError as e:
        print(f"TypeError encountered for method {method}: {e}")
        continue

,Description,Value
0,Session id,123
1,Target,Stance
2,Target type,Multiclass
3,Target mapping,"Opposed: 0, Supports: 1, Supports Beyond Schedule 3: 2, Unclear: 3"
4,Original data shape,"(1000, 6)"
5,Transformed data shape,"(475, 7124)"
6,Transformed train set shape,"(175, 7124)"
7,Transformed test set shape,"(300, 7124)"
8,Ignore features,4
9,Text features,1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7129,0.0000,0.7129,0.7268,0.6789,0.4737,0.5113,2.1410
ridge,Ridge Classifier,0.7071,0.0000,0.7071,0.7296,0.6984,0.4980,0.5110,1.7060
et,Extra Trees Classifier,0.6529,0.7647,0.6529,0.6497,0.6390,0.4070,0.4150,1.7250
svm,SVM - Linear Kernel,0.6329,0.0000,0.6329,0.6556,0.6321,0.3956,0.4033,1.6770
rf,Random Forest Classifier,0.6257,0.7635,0.6257,0.6456,0.6219,0.3837,0.3926,1.6950
nb,Naive Bayes,0.6029,0.6734,0.6029,0.5998,0.5926,0.3316,0.3369,1.9060
lda,Linear Discriminant Analysis,0.6000,0.0000,0.6000,0.6101,0.5839,0.3270,0.3452,1.7370
lightgbm,Light Gradient Boosting Machine,0.5771,0.7468,0.5771,0.6380,0.5821,0.3311,0.3463,1.8060
gbc,Gradient Boosting Classifier,0.5471,0.0000,0.5471,0.6166,0.5541,0.2981,0.3158,2.2520
dummy,Dummy Classifier,0.5357,0.5000,0.5357,0.2870,0.3738,0.0000,0.0000,1.6810


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

,Description,Value
0,Session id,123
1,Target,Stance
2,Target type,Multiclass
3,Target mapping,"Opposed: 0, Supports: 1, Supports Beyond Schedule 3: 2, Unclear: 3"
4,Original data shape,"(1000, 6)"
5,Transformed data shape,"(676, 7124)"
6,Transformed train set shape,"(376, 7124)"
7,Transformed test set shape,"(300, 7124)"
8,Ignore features,4
9,Text features,1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7271,0.0000,0.7271,0.7154,0.6748,0.4707,0.5490,1.5670
lightgbm,Light Gradient Boosting Machine,0.7214,0.2583,0.7214,0.7061,0.6699,0.4626,0.5357,0.5770
ridge,Ridge Classifier,0.7143,0.0000,0.7143,0.7115,0.6580,0.4417,0.5293,0.3010
svm,SVM - Linear Kernel,0.7129,0.0000,0.7129,0.6974,0.6606,0.4451,0.5215,0.3100
knn,K Neighbors Classifier,0.7114,0.2209,0.7114,0.6926,0.6614,0.4575,0.5167,0.2950
rf,Random Forest Classifier,0.7100,0.2553,0.7100,0.7096,0.6527,0.4324,0.5222,0.3160
et,Extra Trees Classifier,0.7100,0.2505,0.7100,0.7096,0.6527,0.4324,0.5222,0.2990
dt,Decision Tree Classifier,0.7071,0.2128,0.7071,0.6745,0.6583,0.4476,0.4993,0.2920
lr,Logistic Regression,0.7029,0.0000,0.7029,0.7063,0.6450,0.4155,0.5082,0.3960
nb,Naive Bayes,0.7029,0.2119,0.7029,0.6941,0.6460,0.4336,0.5054,0.3130


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
svm,SVM - Linear Kernel,0.5957,0.0000,0.5957,0.7603,0.6059,0.4214,0.4665,0.8460
lightgbm,Light Gradient Boosting Machine,0.5886,0.8594,0.5886,0.7557,0.6008,0.4173,0.4618,3.5560
gbc,Gradient Boosting Classifier,0.5829,0.0000,0.5829,0.7595,0.5942,0.4135,0.4623,4.9220
ridge,Ridge Classifier,0.5743,0.0000,0.5743,0.7738,0.6014,0.4077,0.4561,0.7250
dt,Decision Tree Classifier,0.5643,0.7111,0.5643,0.7174,0.5769,0.3814,0.4193,0.6560
knn,K Neighbors Classifier,0.5443,0.7420,0.5443,0.7310,0.5589,0.3551,0.3980,0.6660
rf,Random Forest Classifier,0.5429,0.8365,0.5429,0.7513,0.5587,0.3714,0.4238,0.7260
lr,Logistic Regression,0.5357,0.0000,0.5357,0.7785,0.5701,0.3711,0.4294,0.9420
et,Extra Trees Classifier,0.5314,0.8405,0.5314,0.7529,0.5541,0.3617,0.4160,0.7900
nb,Naive Bayes,0.5271,0.6524,0.5271,0.6077,0.5316,0.2815,0.3025,0.7240


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

,Description,Value
0,Session id,123
1,Target,Stance
2,Target type,Multiclass
3,Target mapping,"Opposed: 0, Supports: 1, Supports Beyond Schedule 3: 2, Unclear: 3"
4,Original data shape,"(1000, 6)"
5,Transformed data shape,"(1796, 7124)"
6,Transformed train set shape,"(1496, 7124)"
7,Transformed test set shape,"(300, 7124)"
8,Ignore features,4
9,Text features,1


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7757,0.9097,0.7757,0.7695,0.7641,0.6081,0.6149,5.5540
ridge,Ridge Classifier,0.7700,0.0000,0.7700,0.7836,0.7672,0.6090,0.6173,0.7210
lr,Logistic Regression,0.7671,0.0000,0.7671,0.8004,0.7709,0.6133,0.6229,1.0210
svm,SVM - Linear Kernel,0.7671,0.0000,0.7671,0.7720,0.7578,0.5996,0.6076,0.9750
gbc,Gradient Boosting Classifier,0.7643,0.0000,0.7643,0.7788,0.7646,0.6037,0.6096,7.6220
rf,Random Forest Classifier,0.7414,0.8804,0.7414,0.7417,0.7151,0.5267,0.5563,0.7250
et,Extra Trees Classifier,0.7386,0.8818,0.7386,0.7400,0.7008,0.5101,0.5552,0.8200
dt,Decision Tree Classifier,0.6829,0.7406,0.6829,0.6922,0.6840,0.4753,0.4776,0.6330
nb,Naive Bayes,0.6114,0.6571,0.6114,0.5915,0.5873,0.3153,0.3212,0.7050
ada,Ada Boost Classifier,0.5971,0.0000,0.5971,0.6350,0.5951,0.3399,0.3508,2.3940


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

# Identify the top 10 accuracy scores

In [20]:
top_10_accuracy = results.nlargest(10, 'Accuracy')

print(top_10_accuracy)

                               Model  Accuracy     AUC  Recall   Prec.  \
140  Light Gradient Boosting Machine    0.7886  0.9073  0.7886  0.7845   
196  Light Gradient Boosting Machine    0.7857  0.9076  0.7857  0.7846   
197              Logistic Regression    0.7829  0.0000  0.7829  0.8002   
126  Light Gradient Boosting Machine    0.7814  0.9114  0.7814  0.7766   
154              SVM - Linear Kernel    0.7814  0.0000  0.7814  0.7761   
198              SVM - Linear Kernel    0.7814  0.0000  0.7814  0.7861   
141              SVM - Linear Kernel    0.7800  0.0000  0.7800  0.7756   
168                 Ridge Classifier    0.7786  0.0000  0.7786  0.7880   
210                 Ridge Classifier    0.7786  0.0000  0.7786  0.7820   
182  Light Gradient Boosting Machine    0.7757  0.9115  0.7757  0.7644   

         F1   Kappa     MCC  TT (Sec)             method  
140  0.7781  0.6303  0.6368     2.866  randomoversampler  
196  0.7751  0.6266  0.6340     4.909    borderlinesmote  
197  0.7

# Identify the top 10 recall scores 

In [21]:
top_10_recall = results.nlargest(10, 'Recall')

print(top_10_recall)

                               Model  Accuracy     AUC  Recall   Prec.  \
140  Light Gradient Boosting Machine    0.7886  0.9073  0.7886  0.7845   
196  Light Gradient Boosting Machine    0.7857  0.9076  0.7857  0.7846   
197              Logistic Regression    0.7829  0.0000  0.7829  0.8002   
126  Light Gradient Boosting Machine    0.7814  0.9114  0.7814  0.7766   
154              SVM - Linear Kernel    0.7814  0.0000  0.7814  0.7761   
198              SVM - Linear Kernel    0.7814  0.0000  0.7814  0.7861   
141              SVM - Linear Kernel    0.7800  0.0000  0.7800  0.7756   
168                 Ridge Classifier    0.7786  0.0000  0.7786  0.7880   
210                 Ridge Classifier    0.7786  0.0000  0.7786  0.7820   
182  Light Gradient Boosting Machine    0.7757  0.9115  0.7757  0.7644   

         F1   Kappa     MCC  TT (Sec)             method  
140  0.7781  0.6303  0.6368     2.866  randomoversampler  
196  0.7751  0.6266  0.6340     4.909    borderlinesmote  
197  0.7

# Identify the top 10 Precision scores 

In [22]:
top_10_prec = results.nlargest(10, 'Prec.')

print(top_10_prec)

                               Model  Accuracy     AUC  Recall   Prec.  \
65          Random Forest Classifier    0.4757  0.8009  0.4757  0.8730   
66            Extra Trees Classifier    0.4714  0.7901  0.4714  0.8675   
58                  Ridge Classifier    0.5257  0.0000  0.5257  0.8574   
57               Logistic Regression    0.5271  0.0000  0.5271  0.8463   
59               SVM - Linear Kernel    0.5186  0.0000  0.5186  0.8351   
60      Gradient Boosting Classifier    0.4943  0.0000  0.4943  0.8080   
62   Light Gradient Boosting Machine    0.4914  0.8023  0.4914  0.8040   
183              Logistic Regression    0.7743  0.0000  0.7743  0.8012   
240              Logistic Regression    0.7671  0.0000  0.7671  0.8004   
197              Logistic Regression    0.7829  0.0000  0.7829  0.8002   

         F1   Kappa     MCC  TT (Sec)                     method  
65   0.5403  0.3276  0.4319     0.684  instancehardnessthreshold  
66   0.5339  0.3240  0.4273     0.669  instancehard

# Identify the top 10 F1 scores

In [23]:
top_10_f1 = results.nlargest(10, 'F1')

print(top_10_f1)

                               Model  Accuracy     AUC  Recall   Prec.  \
197              Logistic Regression    0.7829  0.0000  0.7829  0.8002   
183              Logistic Regression    0.7743  0.0000  0.7743  0.8012   
140  Light Gradient Boosting Machine    0.7886  0.9073  0.7886  0.7845   
196  Light Gradient Boosting Machine    0.7857  0.9076  0.7857  0.7846   
155     Gradient Boosting Classifier    0.7743  0.0000  0.7743  0.7811   
200     Gradient Boosting Classifier    0.7700  0.0000  0.7700  0.7790   
240              Logistic Regression    0.7671  0.0000  0.7671  0.8004   
144              Logistic Regression    0.7671  0.0000  0.7671  0.7954   
143     Gradient Boosting Classifier    0.7686  0.0000  0.7686  0.7804   
156              Logistic Regression    0.7671  0.0000  0.7671  0.7987   

         F1   Kappa     MCC  TT (Sec)             method  
197  0.7851  0.6397  0.6445     0.915    borderlinesmote  
183  0.7797  0.6300  0.6361     0.922             adasyn  
140  0.7